In [ ]:
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer, Date

 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.
     |████████████████████████████████| 43 kB 1.8 MB/s 


In [ ]:
engine = create_engine("mysql+pymysql://root:root@/")
engine.execute("CREATE DATABASE car_rental") #create db
engine.execute("USE car_rental") # select new db

In [ ]:
eng = create_engine("mysql+pymysql://root:root@/car_rental")
base = declarative_base()

class Cars(base):
    __tablename__ = 'cars'

    car_id = Column(Integer, primary_key=True, autoincrement=True)
    producer = Column(String(30), nullable=False)
    model = Column(String(30), nullable=False)
    year = Column(Integer, nullable=False)
    horse_power = Column(Integer, nullable=False)
    price_per_day = Column(Integer, nullable=False)

class Clients(base):
    __tablename__ = 'clients'

    client_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(30), nullable=False)
    surname = Column(String(30), nullable=False)
    address = Column(String(30), nullable=False)
    city = Column(String(30), nullable=False)

class Bookings(base):
    __tablename__ = 'bookings'
    booking_id = Column(Integer, primary_key=True, autoincrement=True)
    client_id = Column(Integer, nullable=False)
    car_id = Column(Integer, nullable=False)
    start_date = Column(Date, nullable=False)
    end_date = Column(Date, nullable=False)
    total_amount = Column(Integer, nullable=False)

base.metadata.create_all(eng)

In [ ]:
query ="""
INSERT INTO cars
VALUES (1,'AUDI','A4','2020','200','1000'),
        (2,'AUDI','A4','2010','150','500'),
        (3,'Mercedes','E123','2002','100','200'),
        (4,'Fiat','Maluch','1990','50','100')
"""
engine.execute(query)

In [ ]:
result = engine.execute("select * from clients")
for row in result:
  print(row)

In [ ]:
import pandas as pd
pd.read_sql('SELECT * FROM cars', con=engine)

,car_id,producer,model,year,horse_power,price_per_day
0,1,AUDI,A4,2020,200,1000
1,2,AUDI,A4,2010,150,500
2,3,Mercedes,E123,2002,100,200
3,4,Fiat,Maluch,1990,50,100


In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=eng)
session = Session()

client_1 = Clients(name='Jan', surname='Kowalski',
                   address='ul. Florianska 12', city='Krakow')
car_1 = Cars(producer='Seat', model='Leon', year=2016,
             horse_power=80, price_per_day=200)

session.add(client_1)
session.add(car_1)
session.commit()

In [ ]:
result = engine.execute("select * from clients")
for row in result:
  print(row)

(1, 'Jan', 'Kowalski', 'ul. Florianska 12', 'Krakow')


In [ ]:
import pandas as pd
pd.read_sql('SELECT * FROM cars', con=engine)

,car_id,producer,model,year,horse_power,price_per_day
0,1,AUDI,A4,2020,200,1000
1,2,AUDI,A4,2010,150,500
2,3,Mercedes,E123,2002,100,200
3,4,Fiat,Maluch,1990,50,100
4,5,Seat,Leon,2016,80,200
